In [3]:
#pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time


import re


In [5]:

def return_mode(soup):
    soup_mode = str(soup.find(class_ = "tw"))
    if re.search(r'full/(.*)_text', soup_mode).group(1) == "s":
        return "Single"
    return "Double"

def return_song_name(soup):
    return soup.find(class_ = "song_name").get_text()

def return_score(soup):
    return soup.find(class_ = "txt_v").get_text()

def return_grade(soup):
    soup_grade = str(soup.find(class_ = "img"))
    return re.search(r'grade/(.*).png', soup_grade).group(1)

def return_plate(soup):
    soup_grade = str(soup.find(class_ = "img st1"))
    return re.search(r'plate/(.*).png', soup_grade).group(1)

def parse_soup_into_record_list(soup):
    if soup.find(class_ = "no_con") != None:
        return []
    soup_full = soup.find(class_ = "my_best_scoreList flex wrap")
    soup_list_raw = soup_full.find_all(class_ = "in")
    ans = []
    if len(soup_list_raw) == 0:
        return []
    for item in soup_list_raw:
        game_mode = return_mode(item)
        song_name = return_song_name(item)
        score = return_score(item)
        grade = return_grade(item)
        plate = return_plate(item)
        ans.append({"game_mode": game_mode, "song_name": song_name, "score": score, "grade": grade, "plate": plate})

    
    return ans

def crawl_record_by_level_unit(driver, level, page):
    driver.get(f'https://piugame.com/my_page/my_best_score.php?lv={level}&&page={page}')
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    record = parse_soup_into_record_list(soup)
    #print(soup.find(class_ = "xi last"))
    hasnext = soup.find(class_ = "xi last") != None
    return record, hasnext

def crawl_record_by_level(driver, level):
    get_next = True
    total_record = []
    page_index = 1
    while(get_next):
        print("level: " + str(level)+ ", current index:" + str(page_index))
        record, get_next = crawl_record_by_level_unit(driver, level, page_index)
        for item in record:
            total_record.append(item)
        time.sleep(5)
        page_index += 1

    return total_record

#returns: song name, score, grade, plate


# Set up the Selenium WebDriver
def selenium_login(ID, password):
    driver = webdriver.Chrome()
    # Navigate to the login page
    driver.get('https://piugame.com')

    # Find the username and password input fields and enter your credentials
    username = driver.find_element(By.NAME, 'mb_id')  # Adjust the name attribute if necessary
    password = driver.find_element(By.NAME, 'mb_password')  # Adjust the name attribute if necessary

    username.send_keys('chorjgml@naver.com')
    password.send_keys('helloworld1!')
    password.send_keys(Keys.RETURN)
    #
    time.sleep(5)
    return driver
    # Submit the login form
    #river.find_element(By.CLASS_NAME, 'btn st1').click()
    # Wait for the page to load (adjust the sleep time if necessary)




In [38]:
def crawl_recent_records(driver):
    driver.get('https://piugame.com/my_page/recently_played.php')
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    return soup



def get_recent_plays_list(soup):
    return soup.find_all(class_ = "in bgfix")

def find_level(soup):
    soupstr = str(soup)
    mode = re.search(r'full/(.*)_text', soupstr).group(1)
    if mode == "d":
        mode_full = "Double"
    elif mode == "s":
        mode_full = "Single"
    else:
        mode_full = "Unknown"
    pattern = r'num_(.*?).png'
    level = re.findall(pattern, soupstr)
    levelstr = level[0] + level[1]
    return mode_full + levelstr

def analyze_recent_plays(souplist):
    songname = []
    scores = []
    is_cleared = []
    levels = []
    for item in souplist:
        song = item.find(class_ = "song_name flex").get_text()
        level = find_level(item)
        songname.append(song + " " + level)
        scores.append(item.find(class_ = "tx").get_text())
        levels.append(find_level(item))
        is_cleared.append("plate" in str(item))
    return songname, scores, is_cleared



In [26]:
def get_recent_results(driver):
    recent_plays = crawl_recent_records(driver)
    test = get_recent_plays_list(recent_plays)
    songname, scores, is_cleared = analyze_recent_plays(test)
    ans = {}
    for i in range(len(songname)):
        if is_cleared == True:
            ans[songname[i]] = scores[i]
        else:
            ans[songname[i]] = "Failed"
    return ans


In [31]:
driver = selenium_login('chorjgml@naver.com', 'helloworld1!')


In [39]:
recent_results = get_recent_results(driver)

Double26
Double26
Unknownxx
Unknownxx
Double22
Double22
Double26
Double26
Double26
Double26
Double25
Double25
Double26
Double26
Double26
Double26
Unknownxx
Unknownxx
Double25
Double25
Unknownxx
Unknownxx
Unknownxx
Unknownxx
Double23
Double23
Double26
Double26
Unknownxx
Unknownxx
Unknownxx
Unknownxx
Double24
Double24
Double24
Double24
Unknownxx
Unknownxx
Double22
Double22
Double22
Double22
Unknownxx
Unknownxx
Double21
Double21
Double26
Double26
Double24
Double24
Double21
Double21
Double25
Double25
Double24
Double24
Double26
Double26
Double22
Double22
Double19
Double19
Double16
Double16
Double25
Double25
Double26
Double26
Double11
Double11
Double25
Double25
Double25
Double25
Double26
Double26
Double21
Double21
Double22
Double22
Double26
Double26
Double22
Double22
Double22
Double22
Double22
Double22
Double20
Double20
Double20
Double20
Double21
Double21
Double20
Double20
Double20
Double20
Double18
Double18


In [40]:
recent_results

{'디그니티 Double26': 'Failed',
 '닥터 엠 Unknownxx': 'Failed',
 '도나텔로 Double22': 'Failed',
 '아클루오이아스 Double26': 'Failed',
 '아발란치 Double25': 'Failed',
 '키메라 Double26': 'Failed',
 '붐!! Unknownxx': 'Failed',
 '쥬핀 Double25': 'Failed',
 '아모르 파티 Double23': 'Failed',
 '스톰 Unknownxx': 'Failed',
 '블루이쉬 로즈 Unknownxx': 'Failed',
 '머독 Double24': 'Failed',
 '야누스 Double22': 'Failed',
 '유포리아닉 Double21': 'Failed',
 '유 앤 아이 Double24': 'Failed',
 '레드 스노우 Double21': 'Failed',
 '캉-캉 ~Orpheus in The Party Mix~ - SHORT CUT - Double25': 'Failed',
 'V3 Double24': 'Failed',
 '롤링 크리스마스 Double22': 'Failed',
 '헬릭스 Double19': 'Failed',
 '유니크 Double16': 'Failed',
 '얼티메이텀 Double25': 'Failed',
 '프ㄹi마 마테ㄹi아 Double26': 'Failed',
 '솔페지에토 Double11': 'Failed',
 '카프리스 오브 디제이 오타다 Double25': 'Failed',
 '리더 Double26': 'Failed',
 '베토벤 바이러스 Double21': 'Failed',
 '붐!! Double22': 'Failed',
 '알레그로 퓨리오소 Double20': 'Failed',
 '솔페지에토 Double20': 'Failed',
 '무혼 2 Double21': 'Failed',
 '설탕음모론 Double20': 'Failed',
 '포가튼 뱀파이어 Double20': 'Failed

In [ ]:
total_record_24 = crawl_record_by_level(driver, 24)
total_record_23 = crawl_record_by_level(driver, 23)
total_record_22 = crawl_record_by_level(driver, 22)
total_record_21 = crawl_record_by_level(driver, 21)
total_record_20 = crawl_record_by_level(driver, 20)



In [50]:
import json
abel_record = {"20": total_record_20, "21": total_record_21, "22": total_record_22, "23": total_record_23, "24": total_record_24, "25": total_record_25}
json.dump(abel_record, open("abel_record.json", "w"), ensure_ascii = False)